In [ ]:
# 1. Setup – GPU + repos + SAM3 checkpoint
import torch, os
print(f"🔥 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ Runtime → Change runtime type → GPU (T4)")

# IMPORTANT: cd to /content BEFORE deleting dirs
%cd /content
!rm -rf /content/sam3 /content/vtrack-ai-studio

# Core deps
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q fastapi uvicorn[standard] python-multipart huggingface_hub requests

# Node 18 via nvm
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.7/install.sh | bash
import subprocess, textwrap
bash_cmd = textwrap.dedent("""
  export NVM_DIR="$HOME/.nvm"
  [ -s "$NVM_DIR/nvm.sh" ] && . "$NVM_DIR/nvm.sh"
  nvm install 18
  nvm alias default 18
  nvm use 18
  node -v
  npm -v
""")
print(subprocess.run(["bash", "-lc", bash_cmd], capture_output=True, text=True).stdout)

# Clone repos
%cd /content
!git clone https://github.com/facebookresearch/sam3
!pip install -e /content/sam3 -q

!git clone https://github.com/Amitro123/vtrack-ai-studio
%cd /content/vtrack-ai-studio/backend
!pip install -r requirements.txt -q

# HF login + SAM3 checkpoint
from google.colab import userdata
from huggingface_hub import login, hf_hub_download

HF_TOKEN = userdata.get("HF_TOKEN")
login(token=HF_TOKEN)

os.makedirs("/content/vtrack-ai-studio/backend/checkpoints/sam3", exist_ok=True)
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir="/content/vtrack-ai-studio/backend/checkpoints/sam3",
    token=HF_TOKEN,
)
print(f"✅ SAM3 checkpoint: {checkpoint_path}")

# Create symlinks for expected checkpoint names
sam3_pt = "/content/vtrack-ai-studio/backend/checkpoints/sam3/sam3.pt"
os.makedirs("/content/vtrack-ai-studio/backend/checkpoints/sam3/checkpoints", exist_ok=True)

for dst in [
    "/content/vtrack-ai-studio/backend/checkpoints/sam3/sam3_hiera_large.pt",
    "/content/vtrack-ai-studio/backend/checkpoints/sam3/checkpoints/sam3_hiera_tiny.pt",
]:
    try:
        if os.path.islink(dst) or os.path.exists(dst):
            os.remove(dst)
        os.symlink(sam3_pt, dst)
        print("✅ linked", dst)
    except OSError as e:
        print("❌ symlink failed:", dst, e)

print("\n✅ Setup complete!")

In [ ]:
# 2. Backend – start FastAPI on port 8000
import subprocess, time, requests

%cd /content/vtrack-ai-studio/backend
!pkill -f uvicorn || true
time.sleep(2)

proc = subprocess.Popen(
    ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"],
)
print(f"🚀 Backend PID: {proc.pid}")

# Health check with retries
for i in range(15):
    try:
        r = requests.get("http://localhost:8000/api/health", timeout=5)
        print("✅ Backend health:", r.json())
        break
    except Exception as e:
        print(f"⏳ Backend not ready yet ({i+1}/15)...")
        time.sleep(3)
else:
    print("❌ Backend failed to start, check logs above.")

In [ ]:
# 3. Frontend (Vite) – runs dev server on port 4173 + Cloudflare tunnel
import subprocess, textwrap, time

bash_cmd = textwrap.dedent("""
  set -e
  cd /content/vtrack-ai-studio

  export NVM_DIR="$HOME/.nvm"
  [ -s "$NVM_DIR/nvm.sh" ] && . "$NVM_DIR/nvm.sh"
  nvm use 18

  echo 'VITE_API_URL=http://localhost:8000' > .env

  echo "NODE VERSION:"; node -v
  echo "NPM VERSION:"; npm -v

  npm install
  nohup npm run dev -- --host 0.0.0.0 --port 4173 > vite.log 2>&1 &
""")

print("🚀 Starting Vite dev server on port 4173...")
subprocess.run(["bash", "-lc", bash_cmd])

# Cloudflare tunnel for UI
!pip install -q cloudflared
!pkill cloudflared || true
!nohup cloudflared tunnel --url http://localhost:4173 > cloudflared_front.log 2>&1 &
time.sleep(10)
!grep -o 'https://[^ ]*\\.trycloudflare\\.com' cloudflared_front.log || echo "Waiting for URL..."
print("\n📡 Copy the URL above and open in your browser – VTrackAI Studio UI")